                                             # End to End Data Analytics project

## >>> Now first import CSV file and explore head

In [3]:
import pandas as pd                                                                        # importing pandas

df = pd.read_csv("C:/Users/rajes/Downloads/customer_shopping_behavior.csv",sep ='\t')      # loading csv file

In [41]:
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually


## >> Now Check Datatype and infomation about file

In [42]:
df.info()                                                                                 # table overview

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             3900 non-null   int64  
 1   Age                     3900 non-null   int64  
 2   Gender                  3900 non-null   object 
 3   Item Purchased          3900 non-null   object 
 4   Category                3900 non-null   object 
 5   Purchase Amount (USD)   3900 non-null   int64  
 6   Location                3900 non-null   object 
 7   Size                    3900 non-null   object 
 8   Color                   3900 non-null   object 
 9   Season                  3900 non-null   object 
 10  Review Rating           3863 non-null   float64
 11  Subscription Status     3900 non-null   object 
 12  Shipping Type           3900 non-null   object 
 13  Discount Applied        3900 non-null   object 
 14  Promo Code Used         3900 non-null   

## >> Now check the summary statistics of all the columns like mean median aggreagations and all using (df.describe())

In [43]:
df.describe()

,Customer ID,Age,Purchase Amount (USD),Review Rating,Previous Purchases
count,3900.000000,3900.000000,3900.000000,3863.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.750065,25.351538
std,1125.977353,15.207589,23.685392,0.716983,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.800000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


## >> Now check the missing values

In [44]:
df.isnull().sum()                                                                       #checking total nulls if it exists

Customer ID                0
Age                        0
Gender                     0
Item Purchased             0
Category                   0
Purchase Amount (USD)      0
Location                   0
Size                       0
Color                      0
Season                     0
Review Rating             37
Subscription Status        0
Shipping Type              0
Discount Applied           0
Promo Code Used            0
Previous Purchases         0
Payment Method             0
Frequency of Purchases     0
dtype: int64

 ### Here you can see there is a column called review rating have some null values
 ### Replace these missing values to *Median* values(category wise)

In [45]:
df['Review Rating'] = df.groupby('Category')['Review Rating'].transform(lambda x:x.fillna(x.median()))     # replacing null values to median within each category

### >> Replaced now check if it still exists

In [46]:
df.isnull().sum()                                                                  # now seems to be all good no null found there

Customer ID               0
Age                       0
Gender                    0
Item Purchased            0
Category                  0
Purchase Amount (USD)     0
Location                  0
Size                      0
Color                     0
Season                    0
Review Rating             0
Subscription Status       0
Shipping Type             0
Discount Applied          0
Promo Code Used           0
Previous Purchases        0
Payment Method            0
Frequency of Purchases    0
dtype: int64

### >> Now Column name seems to be inconstistent transform these into "Snake case"

In [47]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')                    # replacing columns into snake case removing space,lower case and adding unerscore

# As you can see all columns name are seems to be fine now but one which is purchase_amount(USD) remove this last parenthesis thing

df = df.rename(columns = {'purchase_amount_(usd)':'purchase_amount'})

In [48]:
df.columns

Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases'],
      dtype='object')

### >> Segmenting Age group

In [49]:
# now seems to data is fine add some extra columns and segment customers into 4 groups

labels = ['Young Adult','Adult','Middle aged','Senior']           # declaring labels
df['age_group'] = pd.qcut(df['age'],q=4,labels = labels)          # adding column

In [50]:
df[['age','age_group']].head(5)

,age,age_group
0,55,Middle aged
1,19,Young Adult
2,50,Middle aged
3,21,Young Adult
4,45,Middle aged


### >> mapping frequency column (text to numeric) 

In [51]:
# create another column purchase frequency days to know how often they purchase 
# we have only got this data in text format column ,converting into integer format column

frequency_mapping = {
    'Fortnightly' : 14,                # here creating dictionary assigning values
    'Weekly' : 7,
    'Monthly' : 30,
    'Quarterly' : 90,
    'Bi-Weekly' : 14,
    'Annually' : 365,
    'Every 3 Months' : 90
}
# now map text with numeric days

df['frequency_purchase-days'] = df['frequency_of_purchases'].map(frequency_mapping) 




In [52]:
df[['frequency_purchase-days','frequency_of_purchases']].head(5)        # checking altered or not

,frequency_purchase-days,frequency_of_purchases
0,14,Fortnightly
1,14,Fortnightly
2,7,Weekly
3,7,Weekly
4,365,Annually


In [53]:
# we can notice we have 2 columns called Discount applied and promocode means discount applied using promocode
# first find if there any difference b/w them 

df[['discount_applied','promo_code_used']].head(10)                     # the question is both usefull 
(df['discount_applied'] == df['promo_code_used']).all()                 # checking if both equal 'yes it is' then drop one of them

np.True_

df = df.drop('promo_code_used',axis = 1)          # droping column


In [54]:
df.head(5)

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,promo_code_used,previous_purchases,payment_method,frequency_of_purchases,age_group,frequency_purchase-days
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,Middle aged,14
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,Yes,2,Cash,Fortnightly,Young Adult,14
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,Yes,23,Credit Card,Weekly,Middle aged,7
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,Yes,49,PayPal,Weekly,Young Adult,7
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,Yes,31,PayPal,Annually,Middle aged,365


## >>>>> Now data is completely enriched and ready to analysis so here we go

### Connect Notebook with SQL Server 